# Baseline Model Result Viewer

In [1]:
import zipfile
import os
import sqlite3
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from pprint import pprint

In [2]:
# Set campus and Rhino GFA values
actual_gfa = 48940
rhino_gfa = 42348

# Set the metered consumption values at the campus level
metered_dict = {
    'Natural Gas': [1640713.91, 1378197.02, 1437831.29, 1076125.76, 681936.22, 384136.15, 335881.39, 312998.14, 300070.64, 787073.87, 1072845.32, 1315723.20],
    'Electricity': [237115.10, 198155.05, 207163.09, 174719.36, 156527.91, 164800.67, 208726.57, 217642.11, 169641.10, 158094.63, 191244.53, 188406.65],
    'TOE': [1877829.01, 1576352.06, 1644994.39, 1250845.12, 838464.13, 548936.81, 544607.95, 530640.25, 469711.74, 945168.50, 1264089.85, 1504129.85]
}

## Obtain scenario results

In [3]:
def campus_level(key):

    baseline_dir = {
        "UMI": r"C:\Users\sb013698\Desktop\github\Hybrid UBEM Tool\Synthetic Data Generation\Datasets\UMI Files\Baseline",
        "SQLite": r"C:\Users\sb013698\Desktop\github\Hybrid UBEM Tool\Synthetic Data Generation\Datasets\SQLite Files\Baseline",
    }
    
    best_dir = {
        "UMI": r"C:\Users\sb013698\Desktop\github\Hybrid UBEM Tool\Synthetic Data Generation\Datasets\UMI Files\Best",
        "SQLite": r"C:\Users\sb013698\Desktop\github\Hybrid UBEM Tool\Synthetic Data Generation\Datasets\SQLite Files\Best",
    }
    
    if key.lower() == "baseline":
        umi_directory = baseline_dir["UMI"]
        sqlite_directory = baseline_dir["SQLite"]
    elif key.lower() == "best":
        umi_directory = best_dir["UMI"]
        sqlite_directory = best_dir["SQLite"]
    else:
        print("Wrong key!")

    # Set actual and Rhino GFA values of the campus
    actual_gfa = 48940
    rhino_gfa = 42348
    
    # Set the metered consumption values at the campus level
    metered_dict = {
        'Natural Gas': [1640713.91, 1378197.02, 1437831.29, 1076125.76, 681936.22, 384136.15, 335881.39, 312998.14, 300070.64, 787073.87, 1072845.32, 1315723.20],
        'Electricity': [237115.10, 198155.05, 207163.09, 174719.36, 156527.91, 164800.67, 208726.57, 217642.11, 169641.10, 158094.63, 191244.53, 188406.65],
        'TOE': [1877829.01, 1576352.06, 1644994.39, 1250845.12, 838464.13, 548936.81, 544607.95, 530640.25, 469711.74, 945168.50, 1264089.85, 1504129.85]
    }
    # Define the metered consumption data for the selected archetype
    metered_ng = metered_dict["Natural Gas"]
    metered_elec = metered_dict["Electricity"]
    metered_toe = metered_dict["TOE"]
    
    # Initialize lists to store monthly energy consumption per run
    natural_gas_cycles = list()
    electricity_cycles = list()
    toe_cycles = list()
    
    # Iterate over cycles
    num_runs = 10
    
    for run in range(1, num_runs+1):
            
        # Connect to the SQLite database
        conn = sqlite3.connect(os.path.join(sqlite_directory, "C{}_results.sqlite3".format(run)))
    
        # 1) object name
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM object_name_assignment;")
        building_list = [row[1] for row in cursor.fetchall()]
        cursor.close()            
    
        # 2) Series
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM series;")
        end_use_settings = cursor.fetchall()
        end_use_order = [setting[1] for setting in end_use_settings[:8]]
        cursor.close()
    
        # 3) Data Point
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM data_point;")
        data_point_settings = cursor.fetchall()
        cursor.close()
    
        # Close the database
        conn.close()
    
        # Check if end-use order matches the default order
        default_end_use_order = [
            'SDL/Cooling',
            'SDL/Heating',
            'SDL/Domestic Hot Water',
            'SDL/Equipment',
            'SDL/Lighting',
            'SDL/Window Radiation',
            'SDL/Total Operational Energy',
            'SDL Energy/Total Area',
        ]
    
        if end_use_order == default_end_use_order:
    
            # Check if the number of data points is valid
            if len(data_point_settings) != len(building_list) * 85:
                #print("PROBLEM: The number of data points does not match the number of buildings!")
                #break
                raise ValueError("Mismatch in data points and buildings count!")
    
            # Initialize arrays to store monthly energy consumption for the current cycle
            natural_gas = np.zeros(12)
            electricity = np.zeros(12)
            toe = np.zeros(12)
            
            # Loop through each building and calculate energy consumption
            for i in range(1, len(building_list)):
                index_range = np.arange(i * 85, (i + 1) * 85)
                
                cooling = [data_point_settings[i][2] for i in index_range[0:12]]
                heating = [data_point_settings[i][2] for i in index_range[12:24]]
                dhw = [data_point_settings[i][2] for i in index_range[24:36]]
                equipment = [data_point_settings[i][2] for i in index_range[36:48]]
                lighting = [data_point_settings[i][2] for i in index_range[48:60]]
                
                # Update monthly energy consumption for each end-use
                natural_gas += np.array(heating) + np.array(dhw)
                electricity += np.array(cooling) + np.array(equipment) + np.array(lighting)
                toe += np.array(heating) + np.array(dhw) + np.array(cooling) + np.array(equipment) + np.array(lighting)
            
            # Append the cycle's monthly consumption data to the lists
            natural_gas_cycles.append(natural_gas)
            electricity_cycles.append(electricity)
            toe_cycles.append(toe)
        
    # Calculate the average monthly energy consumption over all cycles
    avg_natural_gas = np.mean(natural_gas_cycles, axis=0)
    avg_electricity = np.mean(electricity_cycles, axis=0)
    avg_toe = np.mean(toe_cycles, axis=0)
    
    # Convert average consumption values to energy use intensities (kWh/m2)
    avg_ng_eui = avg_natural_gas / rhino_gfa
    avg_elec_eui = avg_electricity / rhino_gfa
    avg_toe_eui = avg_toe / rhino_gfa
    
    # Calculate monthly scenario errors: CV-RMSE & MAPE
    metered_ng_eui = np.array(metered_ng) / actual_gfa
    ng_mape = mean_absolute_percentage_error(metered_ng_eui, avg_ng_eui)
    ng_cvrmse = np.sqrt(mean_squared_error(metered_ng_eui, avg_ng_eui)) / np.mean(metered_ng_eui)
    
    metered_elec_eui = np.array(metered_elec) / actual_gfa
    elec_mape = mean_absolute_percentage_error(metered_elec_eui, avg_elec_eui)
    elec_cvrmse = np.sqrt(mean_squared_error(metered_elec_eui, avg_elec_eui)) / np.mean(metered_elec_eui)
    
    metered_toe_eui = np.array(metered_toe) / actual_gfa
    toe_mape = mean_absolute_percentage_error(metered_toe_eui, avg_toe_eui)
    toe_cvrmse = np.sqrt(mean_squared_error(metered_toe_eui, avg_toe_eui)) / np.mean(metered_toe_eui)
    
    # Save scenario results to result_dict
    result_dict = {
        'NG_MAPE': ng_mape,
        'NG_CV-RMSE': ng_cvrmse, 
        'Elec_MAPE': elec_mape,
        'Elec_CV-RMSE': elec_cvrmse,
        'TOE_MAPE': toe_mape,
        'TOE_CV-RMSE': toe_cvrmse,
    }
        
    return result_dict

In [4]:
# Define scenarios with descriptive names for readability
scenarios = {
    "Baseline Scenario": "baseline",
    "Best Scenario": "best"
}

# Loop through each scenario and process results
results = {}
for scenario_name, key in scenarios.items():
    
    results[scenario_name] = campus_level(key)
    print(f"Results for {scenario_name}:")
    pprint(results[scenario_name])
    print("\n" + "="*50 + "\n")

Results for Baseline Scenario:
{'Elec_CV-RMSE': 1.925680003088306,
 'Elec_MAPE': 1.9673588268610367,
 'NG_CV-RMSE': 0.5091681454356902,
 'NG_MAPE': 0.5453594547341679,
 'TOE_CV-RMSE': 0.41467844363144496,
 'TOE_MAPE': 0.5602031885045509}


Results for Best Scenario:
{'Elec_CV-RMSE': 0.30829836407940375,
 'Elec_MAPE': 0.3009420992799605,
 'NG_CV-RMSE': 0.2841120390251,
 'NG_MAPE': 0.28062465542754805,
 'TOE_CV-RMSE': 0.21030895728327448,
 'TOE_MAPE': 0.22041164033318458}




# END